When describing the planets of our solar system we often highlight how unique each one is. From the sunscathed surface of Mercury to the deep blue colors of Neptune, each planet has some interesting quality not seen in the others. \
It makes you wonder! \
        - If each planet in our system is charming in their own way, what sets planets of other systems apart from them? \
        - These exo-planets, are they all especially unique? \
        - Do their characteristics eventually fall into some trends? What are those characteristics? \
Unfortunately, some of these questions can't be answered with the technology we possess now, but we can learn some things! \
For years, scientists have observed exo-planets using a variety of techniques to help discern as much as they could about them. \
And in this workbook I'll be showcasing some analysis on the data they've collected.

In [1]:
import astroquery
from astroquery.mast import Catalogs, Observations
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery import open_exoplanet_catalogue as oec
from astroquery.open_exoplanet_catalogue import findvalue
import numpy as np
import pandas as pd
from time import time
from astroquery.exoplanet_orbit_database import ExoplanetOrbitDatabase
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d, Span
from bokeh.io import output_notebook
output_notebook()

from functools import wraps

def measure(func):
    @wraps(func)
    def _time_it(*args, **kwargs):
        start = int(round(time() * 1000))
        try:
            return func(*args, **kwargs)
        finally:
            end_ = int(round(time() * 1000)) - start
            print(f"Total execution time: {end_ if end_ > 0 else 0} ms")
    return _time_it

Loading BokehJS ...

To do analysis I'll be using Python. \
The main portion of the data will come from the Open Exoplanet Catalogue. \
And I'll extract the data using the Astropy/Astroquery Python libraries

In [2]:
data = oec.get_catalogue()

We're all familiar with our solar system, so let's take a look at another one. \
One of the most notable and interesting systems discovered to data: TRAPPIST-1

In [3]:
exo_planet_count = 0
print('STAR:')
for properties in data.findall(".//star[name='TRAPPIST-1']/*"):
    if properties.tag == 'planet':
        exo_planet_count += 1
    else:
        print('\t{}:{}'.format(properties.tag, properties.text))
print('\t' + '# of planets: {}'.format(exo_planet_count))

STAR:
	name:TRAPPIST-1
	name:2MASS J23062928-0502285
	name:K2-112
	name:EPIC 246199087
	name:Gaia DR2 2635476908753563008
	magV:18.798
	magR:16.401
	magI:13.966
	magJ:11.35
	magH:10.72
	magK:10.30
	spectraltype:M8 V
	temperature:2516
	metallicity:0.04
	radius:0.121
	mass:0.089
	age:7.6
	# of planets: 7


This is information about the system's star: TRAPPIST-1!

In [4]:
#Let's get the trappist-1 system's data!
for star in data.findall(".//star[name='TRAPPIST-1']"):
    for properties in star.findall('*'):
        if properties.tag != 'planet':
            print('{}:{}'.format(properties.tag, properties.text))
    print('PLANETS:')
    for planet in star.findall(".//planet"):
        for properties in planet.findall('*'):
            #print('{}:{}'.format(properties.tag, properties.text))
            pass
        if findvalue(planet, 'mass') != None and findvalue(star, 'mass') != None:
            print('\tNAME: ', findvalue(planet, 'name'))
            print('\tMASS MEASURES: ', findvalue(planet, 'mass').machine_readable())
            print('\tRADIUS MESASURES: ', findvalue(star, 'radius').machine_readable())
            pass
        else:
            #print('Null')
            pass

name:TRAPPIST-1
name:2MASS J23062928-0502285
name:K2-112
name:EPIC 246199087
name:Gaia DR2 2635476908753563008
magV:18.798
magR:16.401
magI:13.966
magJ:11.35
magH:10.72
magK:10.30
spectraltype:M8 V
temperature:2516
metallicity:0.04
radius:0.121
mass:0.089
age:7.6
PLANETS:
	NAME:  TRAPPIST-1 b
	MASS MEASURES:  0.0032	0.000485	4.5e-05	None	None
	RADIUS MESASURES:  0.121	0.003	0.003	None	None
	NAME:  TRAPPIST-1 c
	MASS MEASURES:  0.003637	0.000447	0.000412	None	None
	RADIUS MESASURES:  0.121	0.003	0.003	None	None
	NAME:  TRAPPIST-1 d
	MASS MEASURES:  0.000934	0.000122	0.00011	None	None
	RADIUS MESASURES:  0.121	0.003	0.003	None	None
	NAME:  TRAPPIST-1 e
	MASS MEASURES:  0.002429	0.000249	0.000236	None	None
	RADIUS MESASURES:  0.121	0.003	0.003	None	None
	NAME:  TRAPPIST-1 f
	MASS MEASURES:  0.002939	0.000252	0.000245	None	None
	RADIUS MESASURES:  0.121	0.003	0.003	None	None
	NAME:  TRAPPIST-1 g
	MASS MEASURES:  0.003612	0.000308	0.000299	None	None
	RADIUS MESASURES:  0.121	0.003	0.003	Non

This is information on each planet: name, mass, equitorial radius

In [5]:
trappist_data = []
cols =  ['name', 'transittime', 'radius', 'semimajoraxis', 'temperature', 'mass', 'lastupdate']
for star in data.findall(".//star[name='TRAPPIST-1']"):
    for planet in star.findall(".//planet"):
        temp = []
        for column in cols:
            temp.append(planet.find(column).text)
        trappist_data.append(temp)
trappist_data = pd.DataFrame(trappist_data, columns = cols)
trappist_data

,name,transittime,radius,semimajoraxis,temperature,mass,lastupdate
0,TRAPPIST-1 b,2457606.56117,0.10001,0.01154775,400.1,0.003200,18/09/18
1,TRAPPIST-1 c,2457568.58230,0.09769,0.01581512,341.9,0.003637,18/09/18
2,TRAPPIST-1 d,2457682.2921,0.06994,0.02228038,288.0,0.000934,18/09/18
3,TRAPPIST-1 e,2457574.9829,0.08118,0.02928285,251.3,0.002429,18/09/18
4,TRAPPIST-1 f,2457616.1548,0.09332,0.03853361,219.0,0.002939,18/09/18
5,TRAPPIST-1 g,2457529.4724,0.10242,0.04687692,198.6,0.003612,18/09/18
6,TRAPPIST-1 h,2457700.0875,0.06896,0.06193488,167,0.001041,18/09/18


In [6]:
#Let's convert jupiter mass to earth!
j_to_e_ratio_mass = (1.89813*10**27) / (5.9722*10**24)
j_to_e_ratio_radi = 11.209
print(j_to_e_ratio_mass, j_to_e_ratio_radi)
trappist_data['mass'] = trappist_data['mass'].astype(float).multiply(other = j_to_e_ratio_mass)
trappist_data['radius'] = trappist_data['radius'].astype(float).multiply(other = j_to_e_ratio_radi)

317.82760121898133 11.209


In [7]:
trappist_data

,name,transittime,radius,semimajoraxis,temperature,mass,lastupdate
0,TRAPPIST-1 b,2457606.56117,1.121012,0.01154775,400.1,1.017048,18/09/18
1,TRAPPIST-1 c,2457568.58230,1.095007,0.01581512,341.9,1.155939,18/09/18
2,TRAPPIST-1 d,2457682.2921,0.783957,0.02228038,288.0,0.296851,18/09/18
3,TRAPPIST-1 e,2457574.9829,0.909947,0.02928285,251.3,0.772003,18/09/18
4,TRAPPIST-1 f,2457616.1548,1.046024,0.03853361,219.0,0.934095,18/09/18
5,TRAPPIST-1 g,2457529.4724,1.148026,0.04687692,198.6,1.147993,18/09/18
6,TRAPPIST-1 h,2457700.0875,0.772973,0.06193488,167,0.330859,18/09/18


In [8]:
source = ColumnDataSource(data = dict(semi_major = trappist_data['semimajoraxis'].astype(float).tolist(),
                                             mass = trappist_data['mass'],
                                             name = trappist_data['name'].tolist()))

p = figure(title = 'TRAPPIST-1 System', width = 1000, height = 500, x_range=Range1d(0, 0.08))
p.scatter(x = 'semi_major', y = 'mass', source = source, size = 10)
p.xaxis[0].axis_label = 'Semi-Major Axis (AU)'
p.yaxis[0].axis_label = 'Mass (Earth Masses)'
labels = LabelSet(x = 'semi_major', y = 'mass', text = 'name', x_offset=10, y_offset=0, source = source, render_mode = 'canvas')
p.add_layout(labels)


show(p)

In [9]:
#~~~~~~~~~~~~~~~~~~
#~~~~Old method~~~~
#~~~~~~~~~~~~~~~~~~

# c = SkyCoord('23 06 29.383 -05 02 28.59', unit = (u.hourangle, u.deg))
# trappist_data = Catalogs.query_region(c, catalog = 'TIC')
# trappist_data[np.where(trappist_data['TWOMASS'].astype(str) == '23062928-0502285')[0]]

# #Get data off of wikipedia because not all the info is on catalog:
# trappist_lum = trappist_data['lum'][0]
# print(trappist_lum)

# #0.0 will not fly when we try to calculate habitable zones...
# trappist_lum = trappist_lum + .001

# R_inner = 0.75*np.sqrt(trappist_lum)
# R_outer = 1.00*np.sqrt(trappist_lum)
# print(R_inner, R_outer)

In [10]:
#Let's take a look at the habitable zone of the trappist system!
for star in data.findall(".//star[name='TRAPPIST-1']"):
    for properties in star.findall('*'):
        if properties.tag != 'planet':
            print('{}:{}'.format(properties.tag, properties.text))

name:TRAPPIST-1
name:2MASS J23062928-0502285
name:K2-112
name:EPIC 246199087
name:Gaia DR2 2635476908753563008
magV:18.798
magR:16.401
magI:13.966
magJ:11.35
magH:10.72
magK:10.30
spectraltype:M8 V
temperature:2516
metallicity:0.04
radius:0.121
mass:0.089
age:7.6


In [27]:
#https://en.wikipedia.org/wiki/Sun
#https://en.wikipedia.org/wiki/Luminosity
#https://en.wikipedia.org/wiki/TRAPPIST-1#cite_note-6
#https://exoplanetarchive.ipac.caltech.edu/docs/poet_calculations.html
for star in data.findall(".//star[name='TRAPPIST-1']"):
    trap_temp = float(star.find('temperature').text)
    trap_rad = float(star.find('radius').text)
trap_lum = ((trap_temp / 5772) **4) * trap_rad**2
R_inner = 0.75*np.sqrt(trap_lum)
R_outer = 1.77*np.sqrt(trap_lum)
print('TRAPPIST-1:\n\tAbsolute visual luminosity: {}\n\tInner habitable zone: {}\n\tOuter habitable zone: {}'.format(trap_lum, R_inner, R_outer))

TRAPPIST-1:
	Absolute visual luminosity: 0.0005285766790681068
	Inner habitable zone: 0.0172430966469428
	Outer habitable zone: 0.04069370808678501


In [12]:
p2 = p
r_in_plot  = Span(location = R_inner, dimension = 'height', line_color = 'green', line_width = 3)
r_out_plot = Span(location = R_outer, dimension = 'height', line_color = 'green', line_width = 3)

p2.renderers.extend([r_in_plot, r_out_plot])
show(p2)

In [13]:
trappist_data

,name,transittime,radius,semimajoraxis,temperature,mass,lastupdate
0,TRAPPIST-1 b,2457606.56117,1.121012,0.01154775,400.1,1.017048,18/09/18
1,TRAPPIST-1 c,2457568.58230,1.095007,0.01581512,341.9,1.155939,18/09/18
2,TRAPPIST-1 d,2457682.2921,0.783957,0.02228038,288.0,0.296851,18/09/18
3,TRAPPIST-1 e,2457574.9829,0.909947,0.02928285,251.3,0.772003,18/09/18
4,TRAPPIST-1 f,2457616.1548,1.046024,0.03853361,219.0,0.934095,18/09/18
5,TRAPPIST-1 g,2457529.4724,1.148026,0.04687692,198.6,1.147993,18/09/18
6,TRAPPIST-1 h,2457700.0875,0.772973,0.06193488,167,0.330859,18/09/18


In [14]:
#Adding in Earth for reference. We know Earth is habitable, but we're so far away from TRAPPIST-1's. Why is that?
source = ColumnDataSource(data = dict(semi_major = trappist_data['semimajoraxis'].astype(float).tolist() + [1],
                                             mass = trappist_data['mass'].tolist() + [1],
                                             name = [planet[-1] for planet in trappist_data['name'].tolist()] + ['Earth'],
                                             radius = [i/150 for i in trappist_data['radius'].tolist() + [1]]))
p3 = figure(title = 'TRAPPIST-1 System & Earth', width = 2500, height = 500, x_range=Range1d(0, 1.2))
p3.circle(x = 'semi_major', y = 'mass', radius = 'radius', source = source)
p3.xaxis[0].axis_label = 'Semi-Major Axis (AU)'
p3.yaxis[0].axis_label = 'Mass (Earth Masses)'
labels = LabelSet(x = 'semi_major', y = 'mass', text = 'name', x_offset=20, y_offset=0, source = source, render_mode = 'canvas')
p3.add_layout(labels)

r_in_plot  = Span(location = R_inner, dimension = 'height', line_color = 'green', line_width = 3)
r_out_plot = Span(location = R_outer, dimension = 'height', line_color = 'green', line_width = 3)

p3.renderers.extend([r_in_plot, r_out_plot])

show(p3)

In [28]:
#https://en.wikipedia.org/wiki/Sun
#https://en.wikipedia.org/wiki/Luminosity
#https://en.wikipedia.org/wiki/TRAPPIST-1#cite_note-6
#https://exoplanetarchive.ipac.caltech.edu/docs/poet_calculations.html
sun_lum = 1 # Unit of Sol Luminosity
R_inner_sol = 0.75*np.sqrt(sun_lum)
R_outer_sol = 1.77*np.sqrt(sun_lum)
print('Sol:\n\tAbsolute visual luminosity: {}\n\tInner habitable zone: {}\n\tOuter habitable zone: {}'.format(sun_lum, R_inner_sol, R_outer_sol))
print('TRAPPIST-1:\n\tAbsolute visual luminosity: {}\n\tInner habitable zone: {}\n\tOuter habitable zone: {}'.format(trap_lum, R_inner, R_outer))

Sol:
	Absolute visual luminosity: 1
	Inner habitable zone: 0.75
	Outer habitable zone: 1.77
TRAPPIST-1:
	Absolute visual luminosity: 0.0005285766790681068
	Inner habitable zone: 0.0172430966469428
	Outer habitable zone: 0.04069370808678501


In [29]:
r_in_plot  = Span(location = R_inner_sol, dimension = 'height', line_color = 'green', line_width = 3, line_dash = 'dashed')
r_out_plot = Span(location = R_outer_sol, dimension = 'height', line_color = 'green', line_width = 3, line_dash = 'dashed')

p3.renderers.extend([r_in_plot, r_out_plot])
p3.x_range = Range1d(0, 2)

show(p3)

### Why is our system's HZ different than TRAPPIST-1's?
The habitable zone of a star, at a basic level, is dictated by its star's luminosity (or how much energy it radiates per interval of time). \
Our sun is ~2000x brighter than TRAPPIST-1, so it makes sense that ours looks much different! \
There are other factors that go into calculating a HZ, but this is a good start. With this knowledge let's look at other HZs!

In [20]:
def hz_calc(star_name):
    star_data = data.findall(".//star[name='{}']".format(star_name))[0]
    star_temp = float(findvalue(star_data, 'temperature').value)
    star_rad = float(findvalue(star_data, 'radius').value)
    star_lum = ((star_temp / 5772)**4) * star_rad**2
    return [0.75*np.sqrt(star_lum), 1.77*np.sqrt(star_lum)]

In [45]:
all_exo_data = []
cols =  ['star', 'hz_in', 'hz_out', 'name',  'transittime', 'radius', 'semimajoraxis', 'temperature', 'mass', 'lastupdate']
count = 0
for star in data.findall(".//star"):
    for planet in star.findall(".//planet"):
        count += 1
        temp = []
        temp.append(findvalue(star, 'name'))
        try:
            hz_info = hz_calc(findvalue(star, 'name'))
            temp.append(hz_info[0]); temp.append(hz_info[1])
        except AttributeError:
            temp.append(None); temp.append(None)
        for column in cols[3:]:
            try:
                temp.append(planet.find(column).text)
            except AttributeError:
                temp.append(None)
        all_exo_data.append(temp)
all_exo_data = pd.DataFrame(all_exo_data, columns = cols)
all_exo_data['mass'] = all_exo_data['mass'].astype(float).multiply(other = j_to_e_ratio_mass)
all_exo_data['radius'] = all_exo_data['radius'].astype(float).multiply(other = j_to_e_ratio_radi)
fail_HZ_df = all_exo_data[all_exo_data['hz_in'].isna() | all_exo_data['semimajoraxis'].isna()]
print('Couldn\'t calculate HZ for {} planets or planet did not have semimajoraxis. Stored as fail_HZ_df'.format(len(fail_HZ_df)))
all_exo_data = all_exo_data[~all_exo_data['hz_in'].isna() & ~all_exo_data['semimajoraxis'].isna()]
print('Calculated HZ for {}/{} planets!'.format(len(all_exo_data), count))
all_exo_data['hz_bool'] = all_exo_data.apply(lambda row: row['hz_in'] <= float(row['semimajoraxis']) <= row['hz_out'], axis = 1)

Couldn't calculate HZ for 2977 planets or planet did not have semimajoraxis. Stored as fail_HZ_df
Calculated HZ for 1849/4826 planets!


In [55]:
all_exo_data['hz_bool_color'] = all_exo_data['hz_bool'].map({True: 'crimson', False:'royalblue'})
source = ColumnDataSource(data = dict(semi_major = all_exo_data['semimajoraxis'].astype(float).tolist(),
                                             mass = all_exo_data['mass'],
                                             name = all_exo_data['name'].tolist(),
                                             color = all_exo_data['hz_bool_color']))
p = figure(title = 'Exoplanets & HZs', width = 1500, height = 1000, y_axis_type = 'log', y_range = [10**-3, 10**5], x_axis_type = 'log', x_range = [10**-3, 10**4])
p.scatter(x = 'semi_major', y = 'mass', color = 'color', source = source, size = 3)
p.xaxis[0].axis_label = 'Semi-Major Axis (AU)'
p.yaxis[0].axis_label = 'Mass (Earth Masses)'
show(p)